In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("data.csv", skiprows = 41)

In [2]:
#Deleting invalid data

df = df.drop(df[(df.PSAL < 2)].index)
df = df.drop(df[(df.PSAL > 41)].index)
#len(df)

In [3]:
df = df.drop(df[(df.CPHL < 0)].index)
df = df.drop(df[(df.CPHL > 100)].index)
#len(df)

In [4]:
df = df.drop(df[(df.CDOM < 0)].index)
df = df.drop(df[(df.CDOM > 400)].index)
#len(df)

In [5]:
df = df.drop(df[(df.VBSC < 0)].index)
df = df.drop(df[(df.VBSC > 0.1)].index)
#len(df)

In [6]:
df = df.drop(df[(df.IRRAD555 < 0) ].index)
df = df.drop(df[(df.IRRAD555 > 1000)].index)
#len(df)

In [7]:
df['PSAL']

0          35.3439
1          35.3442
2          35.3441
3          35.3442
4          35.3441
            ...   
3123112    35.2296
3123113    35.2294
3123114    35.2296
3123115    35.2298
3123116        NaN
Name: PSAL, Length: 3101188, dtype: float64

In [8]:
df.isnull().sum()

FID                                0
file_id                            0
deployment_name                    0
platform_type                      0
platform_code                      1
time_coverage_start                1
time_coverage_end                  1
TIME                               1
TIME_quality_control               1
LATITUDE                        2077
LATITUDE_quality_control           1
LONGITUDE                       2077
LONGITUDE_quality_control          1
PRES                            6895
PRES_quality_control               1
DEPTH                           7687
DEPTH_quality_control              1
PROFILE                         1556
PROFILE_quality_control            1
PHASE                           7049
PHASE_quality_control              1
TEMP                            1410
TEMP_quality_control               1
PSAL                            7252
PSAL_quality_control               1
DOX1                          641383
DOX1_quality_control               1
D

In [134]:
# df_cc= df[['LATITUDE','LONGITUDE','UCUR']].dropna(axis=0)

In [139]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import cross_val_score
# knn = KNeighborsClassifier(3, weights = 'distance')
# X = df_cc.loc[:,['LATITUDE','LONGITUDE']]
# y = df_cc.loc[:,'UCUR'].astype('int')
# model_3nn = knn.fit(X,y)
# missing_UCUR = df['UCUR'].isnull()
# df_missing_UCUR = pd.DataFrame(df[['LATITUDE','LONGITUDE']].dropna(axis=0)[missing_UCUR])
# imputed_UCUR = model_3nn.predict(df_missing_UCUR)





/Users/chengqian/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [8]:
# UCUR_obs = df['UCUR'].dropna()
# UCUR_mis = df['UCUR'][missing_UCUR]
# UCUR_3nn_imp = pd.DataFrame(imputed_UCUR)
# UCUR_3nn_imp = UCUR_3nn_imp.T.squeeze()
# mode_UCUR = df['UCUR'].mode()
# df_mode4UCUR = df.fillna(value = {'UCUR':mode_UCUR[0]})
# UCUR_mode_imp = df_mode4UCUR[['UCUR']][missing_UCUR]
# UCUR_mode_imp = UCUR_mode_imp.T.squeeze()
# legend_1 = ['observed UCUR','missing UCUR']
# legend_2 = ['observed UCUR','3nn imputed UCUR']
# legend_3 = ['observed UCUR','mode imputed UCUR']

# plt.subplot(1,3,1)
# plt.hist([UCUR_obs,UCUR_mis],color=['gold','limegreen'])
# plt.xlabel("UCUR")
# plt.ylabel("Frequency")
# plt.legend(legend_1)

# plt.subplot(1,3,2)
# plt.hist([UCUR_obs,UCUR_3nn_imp],color=['gold','royalblue'])
# plt.xlabel("UCUR")
# plt.ylabel("Frequency")
# plt.legend(legend_2)

# plt.subplot(1,3,3)
# plt.hist([UCUR_obs,UCUR_mode_imp],color=['gold','violet'])
# plt.xlabel("UCUR")
# plt.ylabel("Frequency")
# plt.legend(legend_3)

In [8]:
# corr plot for each variable, select those with corr>0.5 as training features
corr = df.corr() 
corr[(corr.abs()>0.5) & (corr.abs() < 1.0)].style.background_gradient(cmap='coolwarm')

/Users/chengqian/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Select features for knn model
df1= df[['PSAL','PRES','DEPTH']].dropna(axis=0)

# Features with corr>0.5
X = df1[['PRES','DEPTH']]
# Target value
y = df1['PSAL'].values

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Import knn 
from sklearn.neighbors import KNeighborsClassifier

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3)

# Fit the classifier to the data
model = knn.fit(X_train,y_train.astype('int'))
#knn.predict(X_test)[0:5]

y_pred = knn.predict(X_test)

# Test the accuracy score of the model
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test.astype('int'), y_pred))

0.9464205756886555


In [11]:
# Apply grid search to find the k value that gives us the best performance
from sklearn.model_selection import GridSearchCV
#create new a knn model
knn2 = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 5)}
#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
#fit model to data
knn_gscv.fit(X, y.astype('int'))
#check top performing n_neighbors value
knn_gscv.best_params_

{'n_neighbors': 3}

In [10]:
missing_PSAL = df['PSAL'].isnull()
df_missing_PSAL = pd.DataFrame(df[['PRES','DEPTH']].fillna(df1.mean())[missing_PSAL])
imputed_PSAL = model.predict(df_missing_PSAL)
imputed_PSAL
#df_out = pd.merge(df,imputed_PSAL,how = 'left',left_index = True, right_index = True)

array([35, 35, 35, ..., 35, 35, 35])

In [20]:
# from sklearn.impute import KNNImputer
# missing_PSAL = df['PSAL'].isnull()
# df_missing_PSAL = pd.DataFrame(df[['PRES','DEPTH','PSAL']][missing_PSAL])
# imputer = KNNImputer(n_neighbors=3)
# df_missing_PSAL = imputer.fit_transform(df_missing_PSAL)
# df_missing_PSAL

In [ ]:
# knn = KNeighborsClassifier(3, weights = 'distance')
# X = df_cc.loc[:,['LATITUDE','LONGITUDE','PRES','DEPTH','PROFILE','PHASE','TEMP','PSAL','CNDC']]
# y = df_cc.loc[:,'HEAD'].astype('int')
# model_3nn = knn.fit(X,y)
# missing_HEAD = df['HEAD'].isnull()
# df_missing_HEAD = pd.DataFrame(df[['LATITUDE','LONGITUDE','PRES','DEPTH','PROFILE','PHASE','TEMP','PSAL','CNDC']].fillna(df_cc.mean())[missing_HEAD])
# imputed_HEAD = model_3nn.predict(df_missing_HEAD)

# HEAD_obs = df['HEAD'].dropna()
# HEAD_mis = df['HEAD'][missing_HEAD]
# HEAD_3nn_imp = pd.DataFrame(imputed_HEAD)
# HEAD_3nn_imp = HEAD_3nn_imp.T.squeeze()
# mode_HEAD = df['HEAD'].mode()
# df_mode4HEAD = df.fillna(value = {'HEAD':mode_HEAD[0]})
# HEAD_mode_imp = df_mode4HEAD[['HEAD']][missing_HEAD]
# HEAD_mode_imp = HEAD_mode_imp.T.squeeze()
# legend_1 = ['observed HEAD','missing HEAD']
# legend_2 = ['observed HEAD','3nn imputed HEAD']
# legend_3 = ['observed HEAD','mode imputed HEAD']

# plt.subplot(1,3,1)
# plt.hist([HEAD_obs,HEAD_mis],color=['gold','limegreen'])
# plt.xlabel("HEAD")
# plt.ylabel("Frequency")
# plt.legend(legend_1)

# plt.subplot(1,3,2)
# plt.hist([HEAD_obs,HEAD_3nn_imp],color=['gold','royalblue'])
# plt.xlabel("HEAD")
# plt.ylabel("Frequency")
# plt.legend(legend_2)

# plt.subplot(1,3,3)
# plt.hist([HEAD_obs,HEAD_mode_imp],color=['gold','violet'])
# plt.xlabel("HEAD")
# plt.ylabel("Frequency")
# plt.legend(legend_3)

In [32]:
# df1 = df[df['IRRAD443'].notnull()]
# X = df1['IRRAD443'].values
# y = df1['IRRAD443_quality_control'].values
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)
# X_train = np.array(X_train).reshape(-1, 1)
# y_train = np.array(y_train).reshape(-1, 1)
# X_test = np.array(X_test).reshape(-1, 1)

In [33]:
# from sklearn.ensemble import RandomForestRegressor

# regressor = RandomForestRegressor(n_estimators=20, random_state=0)
# regressor.fit(X_train, y_train.ravel())
# y_pred = regressor.predict(X_test)
# y_pred

In [34]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print(confusion_matrix(y_test,y_pred.round()))
# print(classification_report(y_test,y_pred.round()))
# print(accuracy_score(y_test, y_pred.round()))

In [35]:
# from sklearn.ensemble import RandomForestClassifier
# df1 = df[df['IRRAD443'].notnull()]
# X = df1['IRRAD443'].values
# y = df1['IRRAD443_quality_control'].values
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)
# clf = RandomForestClassifier(max_depth=2, random_state=0)
# X_train = np.array(X_train).reshape(-1, 1)
# y_train = np.array(y_train).reshape(-1, 1)
# X_test = np.array(X_test).reshape(-1, 1)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)



In [36]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))
# print(accuracy_score(y_test, y_pred))